In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

import requests
import collections
import json
import pandas as pd
import numpy as np
import re
import sys

import wikipedia
#https://stackoverflow.com/questions/8088226/content-of-infobox-of-wikipedia

In [2]:
GEO_TIME = "geological_time.csv"
gt = pd.read_csv(GEO_TIME)

def trynum(n):
    try:
        return float(n)
    except:
        return None

In [3]:
d, box = ph.get_dates("Monotremata", gt)

In [4]:
re.sub("[^A-Za-z]+", " ", box).lower()

' automatic taxobox name monotremes ref name msw msw groves pages id ref fossil range late triassic ref cite journal last hugall first a f year title calibration choice rate smoothing and the pattern of tetrapod diversification according to the long nuclear gene rag url journal syst biol volume issue pages doi display authors etal ref holocene fossil range image prototheria collage png image caption a short beaked echidna a platypus a steropodon reconstruction and a western long beaked echidna taxon monotremata authority charles lucien bonaparte c l bonaparte ref cite journal last bonaparte first c l title a new systematic arrangement of vertebrated animals journal transactions of the linnean society of london date volume page doi j tb x url https www biodiversitylibrary org page ref subdivision ranks subgroups subdivision kryoryctes steropodontidae platypoda ornithorhynchidae tachyglossa echidna tachyglossidae monotremes are one of the three main groups of living mammal s along with p

In [37]:
this_query = "Monotremata"
#def get_dates(this_query, gt):
"""gets the earliest and latest date for an organism
this_query - a text sting organism name
gt - the geological time dataframe
"""
try:
    # get wikipedia page
    this_page_id = wikipedia.search(this_query, results=1)[0]

    # fetch infobox
    wiki_url = 'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles={}&rvsection=0'.format(this_page_id)
    box_raw = json.loads(requests.get(wiki_url).content.decode('utf-8'))
    box = next(iter(box_raw['query']['pages'].values()))['revisions'][0]['*']

    # parse out fossil range
    fr = box.split("fossil_range")[1].split('\n')[0]
    useful = re.sub("[^A-Za-z0-9\.]+", " ", fr)
    useful = useful.replace("Late ","Late_").replace("Middle ","Middle_").replace("Early ", "Early_").split(" ")

    # try to get actual MYA times
    times = [trynum(n) for n in useful]
    times = [n for n in times if n is not None]
    if len(times) >= 2:
        output_dates = (max(times), min(times))
    else:
        # parse from periods
        dates = []
        for u in useful:
            ru = u.replace("Late_","").replace("Middle_","").replace("Early_", "")
            try:
                this_row = list(gt[gt['Period']==ru].iloc[0])
                if "Late_" in u:
                    this_dates = [this_row[2]]
                elif "Middle_" in u:
                    this_dates = [np.mean(this_row[1:3])]
                elif "Early_" in u:
                    this_dates = [this_row[1]]
                else:
                    this_dates = this_row[1:3]
                dates += this_dates
            except:
                pass
        output_dates = (max(dates), min(dates))
except:
    output_dates = (None, None)
print(output_dates)

(1.0635150701477824e+16, 0.0)


In [6]:
this_query = "Monotremata"
#def get_dates(this_query, gt):
"""gets the earliest and latest date for an organism
this_query - a text sting organism name
gt - the geological time dataframe
"""

# get wikipedia page
this_page_id = wikipedia.search(this_query, results=1)[0]

# fetch infobox
wiki_url = 'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=json&titles={}&rvsection=0'.format(this_page_id)
box_raw = json.loads(requests.get(wiki_url).content.decode('utf-8'))
box = next(iter(box_raw['query']['pages'].values()))['revisions'][0]['*']
box

'{{automatic taxobox\n| name = Monotremes<ref name=msw3>{{MSW3 Groves|pages=1-2|id=10300001}}</ref>\n| fossil_range = [[Late Triassic]]<ref>{{cite journal | last1 = Hugall | first1 = A.F.| year = 2007 | title = Calibration choice, rate smoothing, and the pattern of tetrapod diversification according to the long nuclear gene RAG-1 | url = | journal = Syst. Biol. | volume = 56 | issue = 4| pages = 543–63 | doi=10.1080/10635150701477825|display-authors=etal}}</ref>–[[Holocene]], {{Fossil range|210|0}}\n| image = Prototheria collage.png\n| image_caption = A [[short-beaked echidna]], a [[platypus]], a \'\'[[Steropodon]]\'\' reconstruction and a [[western long-beaked echidna]]\n| taxon = Monotremata\n| authority = [[Charles Lucien Bonaparte|C.L. Bonaparte]], 1837<ref>{{cite journal|last1=Bonaparte|first1=C. L.|title=A New Systematic Arrangement of Vertebrated Animals|journal=Transactions of the Linnean Society of London|date=1837|volume=18|page=258|doi=10.1111/j.1095-8339.1838.tb00177.x|url=

In [15]:
the_clade = "Australosphenida"
the_clade in box_raw

False

In [14]:
new_box = next(iter(box_raw['query']['pages'].values()))['revisions'][0]['*']
print(new_box[:20])
the_clade in new_box

{{automatic taxobox



False

In [7]:
# parse out fossil range
fr = box.split("fossil_range")[-1].split('\n')[0]#.split('<ref>')[0].split('{{')[0]
print(fr)
useful = re.sub("[^A-Za-z0-9\.]+", " ", fr)
useful = useful.replace("Late ","Late_").replace("Middle ","Middle_").replace("Early ", "Early_").split(" ")
useful

 = [[Late Triassic]]<ref>{{cite journal | last1 = Hugall | first1 = A.F.| year = 2007 | title = Calibration choice, rate smoothing, and the pattern of tetrapod diversification according to the long nuclear gene RAG-1 | url = | journal = Syst. Biol. | volume = 56 | issue = 4| pages = 543–63 | doi=10.1080/10635150701477825|display-authors=etal}}</ref>–[[Holocene]], {{Fossil range|210|0}}


['',
 'Late_Triassic',
 'ref',
 'cite',
 'journal',
 'last1',
 'Hugall',
 'first1',
 'A.F.',
 'year',
 '2007',
 'title',
 'Calibration',
 'choice',
 'rate',
 'smoothing',
 'and',
 'the',
 'pattern',
 'of',
 'tetrapod',
 'diversification',
 'according',
 'to',
 'the',
 'long',
 'nuclear',
 'gene',
 'RAG',
 '1',
 'url',
 'journal',
 'Syst.',
 'Biol.',
 'volume',
 '56',
 'issue',
 '4',
 'pages',
 '543',
 '63',
 'doi',
 '10.1080',
 '10635150701477825',
 'display',
 'authors',
 'etal',
 'ref',
 'Holocene',
 'Fossil',
 'range',
 '210',
 '0',
 '']

In [17]:
# try to get actual MYA times
times = [trynum(n) for n in useful]
times = [n for n in times if n is not None]
if len(times) >= 2:
    output_dates = (max(times), min(times))
else:
    # parse from periods
    dates = []
    for u in useful:
        ru = u.replace("Late_","").replace("Middle_","").replace("Early_", "")
        try:
            this_row = list(gt[gt['Period']==ru].iloc[0])
            if "Late_" in u:
                this_dates = [this_row[2]]
            elif "Middle_" in u:
                this_dates = [np.mean(this_row[1:3])]
            elif "Early_" in u:
                this_dates = [this_row[1]]
            else:
                this_dates = this_row[1:3]
            dates += this_dates
        except:
            pass
    output_dates = (max(dates), min(dates))

print(output_dates)

(419.19999999999999, 419.19999999999999)


# building links_dict

In [25]:
df = pd.read_csv("data/Chordata-2499-noage.csv")
df.head()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y
0,2466,1,0,2499,Chordata,2,0,-0.794184,0.918448
1,2499,2,2,14819,Calcichordata,0,0,-0.091174,0.033661
2,2499,2,0,14821,Urochordata,8,0,-1.014761,0.366714
3,14821,3,0,114489,Octacnemidae,0,0,-0.066112,-0.370996
4,14821,3,0,114490,Phlebobranchiata,4,0,-0.344893,-0.201929


In [26]:
#df = df.loc[df['Begin'].notnull()]
#df = df.loc[df['Begin'] < 500]
print(df.shape)

(10395, 9)


In [13]:
this_id = 15963

In [14]:
this_row = df[df['id'] == this_id]
this_row

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
606,15997,12,0,15963,Primates,2,0,-5.165991,0.69956,55.0,0.0


In [15]:
list(this_row['ancestor'])

[15997]

In [27]:
links_dict = {}
for i, this_row in df.iterrows():
    this_id = int(this_row['id'])
    # get parents
    if isinstance(this_row['ancestor'], int):
        parents = [this_row['ancestor']]
    else:
        parents = list(this_row['ancestor'])
        print(this_row)
    # add parents
    try:
        # add parents if the id already exists
        links_dict[this_id]['parents'] += parents
        # filter to only unique
        links_dict[this_id]['parents'] = list(set(links_dict[this_id]['parents']))
    except KeyError:
        # add this_id
        links_dict[this_id] = {}
        # add parent to this_id
        links_dict[this_id]['parents'] = parents
        links_dict[this_id]['children'] = []

    # add as child to parent node
    for parent in parents:
        try:
            # add as child if parent node already exists
            links_dict[parent]['children'].append(this_id)
        except KeyError:
            # if not, create parent node and add as child
            links_dict[parent] = {}
            links_dict[parent]['children'] = [this_id]
            links_dict[parent]['parents'] = []
        # filter to only unique
        links_dict[parent]['children'] = list(set(links_dict[parent]['children']))


{'parents': [2466], 'children': []}
{'parents': [16015], 'children': []}
{'parents': [100852], 'children': []}
{'parents': [84686], 'children': []}
{'parents': [67993], 'children': []}
{'parents': [66413], 'children': []}
{'parents': [57823], 'children': []}
{'parents': [92006], 'children': []}
{'parents': [92095], 'children': []}
{'parents': [16958], 'children': []}
{'parents': [68930], 'children': []}


In [29]:
len(links_dict.keys())

10396

In [20]:
isinstance(this_row['ancestor'], int)

True

In [ ]:
def create_plot_data(self, 
                 highlight = 'all',
                 max_nodes = 5000,
                 max_depth = 20,
                 color_attr = 'extinct',
                 Z_dim = 'depth',
                 Z_dim_mult = -1,
                 add_links = False):
''''''

In [ ]:
if highlight == 'all':
    pick = self.df.ix[0, 'id']
else:
    pick = highlight

# subset to max_nodes and max_depth
df = self.df[self.df['depth'] <= max_depth].head(max_nodes)
df = df.reset_index()

# get kinfolk
kin = [pick]
# add parents
kin += self.links_dict[pick]['parents']
# add kids
for c in self.links_dict[pick]['children']:
    kin += self.links_dict[c]['children']
kin = list(set(kin))

# subset
df['kin'] = 0
df['kin'][df['id'].isin(kin)] = 1
print(df[df['kin']==1])

###############
### NEED TO CREATE LINKS_LIST HERE
## create links list
links_list = []
for i, n in df.iterrows():
    links_list.append({
        'source': n['id'], 
        'target': n['ancestor'], 
        'value': n['num_kids']
    })

#print("len(links_list): {}".format(len(links_list)))
#print("first 3 links: {}".format(links_list[:3]))

#links_list = [l for l in links_list if l['target'] in list(df['id'])]
L=len(links_list)
#print("len(links_list): {}".format(L))

#
Edges=[(links_list[k]['source'], links_list[k]['target']) for k in range(L)]
#print("Edges {}".format(Edges[:5]))

# 
labels=[]
group=[]
alpha = []
layt = []
for i, node in df.iterrows():
    # create text labels
    if add_links:
        try:
            #this_page_id = wikipedia.search(node['name'], results=1)[0]
            labels.append('<a href="https://en.wikipedia.org/wiki/{}">{}</a>'.format(node['name'], node['name']))
        except:
            labels.append(str(node['name']))
    else:
        labels.append(str(node['depth']) + ' ' + node['name'])
    # create color key
    group.append(node[color_attr])
    # create opacity key
    alpha.append(node['kin'])
    # create layout list
    d = node[Z_dim]
    # # PCA
    # # layt.append(list(X_pca[i]) + [d * -1])
    layt.append([node['x'], 
                 node['y'], 
                 (d*Z_dim_mult)+np.random.uniform(-0.1,0.1,1)[0]])    


#print(labels[:3])
#print(group[:3])
#print(alpha[:3])
#print(len(layt))
#print(layt[:3])

# make nodes
Xn=[layt[k][0] for k in range(len(layt))]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(len(layt))]# y-coordinates
Zn=[layt[k][2] for k in range(len(layt))]# z-coordinates

# check if they're kinfolk
Xnk=[layt[k][0] for k in range(len(layt)) if alpha[k]==1]# x-coordinates of nodes
Ynk=[layt[k][1] for k in range(len(layt)) if alpha[k]==1]# y-coordinates
Znk=[layt[k][2] for k in range(len(layt)) if alpha[k]==1]# z-coordinates
groupk=[group[k] for k in range(len(layt)) if alpha[k]==1]# color
#print(len(Xnk))

# make edges
Xe=[]
Ye=[]
Ze=[]
Xek=[]
Yek=[]
Zek=[]
for e in Edges:
    try: 
        e0 = df[df['id'] == e[0]].index.values[0]
    except:
        e0 = 0
    try:
        e1 = df[df['id'] == e[1]].index.values[0]
    except:
        e1 = 0
    #
    Xe+=[layt[e0][0],layt[e1][0], None]# x-coordinates of edge ends
    Ye+=[layt[e0][1],layt[e1][1], None]  
    Ze+=[layt[e0][2],layt[e1][2], None] 
    # check if they're kinfolk
    if e[0] in kin:
        Xek+=[layt[e0][0],layt[e1][0], None]
        Yek+=[layt[e0][1],layt[e1][1], None]
        Zek+=[layt[e0][2],layt[e1][2], None]